<a href="https://colab.research.google.com/github/brahmbhatt-me/drone_racing_project/blob/main/drone_racing_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚁 **Autonomous Drone Racing with Trajectory Optimization**

**Author:** Meet B  
**Program:** Master's in Robotics, Northeastern University  
**GitHub:** https://github.com/brahmbhatt-me ,
**LinkedIn:** https://www.linkedin.com/in/meet-m-brahmbhatt

---

## 📋 **Project Overview**

Welcome to this comprehensive drone racing simulation project! We'll build a complete autonomous drone system that can navigate through aerial hoops while avoiding obstacles, using state-of-the-art path planning and trajectory optimization techniques.

---

## 🎯 **What We're Building**

A drone racing simulation where the drone must fly through aerial hoops while avoiding obstacles. The complete pipeline includes:

| Step | Module | Description |
|:----:|--------|-------------|
| 1 | **Path Planning** | RRT/RRT* algorithms to find collision-free paths |
| 2 | **Drone Dynamics** | Realistic physics with thrust, drag, gravity |
| 3 | **Trajectory Optimization** | Smooth paths into efficient trajectories |
| 4 | **3D Visualization** | Interactive plotting of results |

---

## 🌍 **Real-World Applications**

This same technology powers:

| Industry | Application |
|----------|-------------|
| 📦 Logistics | Amazon/DJI delivery drones |
| 🔍 Emergency | Search-and-rescue operations |
| 🏎️ Sports | Professional drone racing (DRL, MultiGP) |
| 🎬 Film | Autonomous cinematography |
| 🚀 Aerospace | SpaceX rocket landing trajectories |

---

## 🧠 **Key Concepts You'll Learn**

| Concept | Description |
|---------|-------------|
| **6-DOF State** | Position (x, y, z) + Attitude (yaw, pitch, roll) |
| **Terminal Velocity Model** | Simplified dynamics suitable for planning |
| **RRT / RRT*** | Sampling-based path planning algorithms |
| **Direct Transcription** | Converting optimal control to nonlinear programming |
| **Constrained Optimization** | Satisfying physics while minimizing cost |

---

## 🔄 **Project Pipeline**
```
[Start] → [RRT Planning] → [Trajectory Optimization] → [3D Visualization] → [Done]
```

---

## 📁 **Project Structure**
```
drone_racing/
├── dynamics/          # Drone physics model
│   ├── drone_model.py
│   └── kinematics.py
├── planning/          # Path planning algorithms
│   ├── rrt.py
│   └── collision.py
├── optimization/      # Trajectory optimization
│   ├── optimizer.py
│   ├── cost_functions.py
│   └── constraints.py
└── visualization/     # 3D plotting
    └── plotter.py
```

---

## ▶️ **Let's Begin!**

Proceed to **Step 1** to install the required dependencies.

#  Setup and Dependencies

This section prepares the Colab environment for running the full drone racing pipeline.  
We will install the required libraries, set up numerical tools, and import helper modules used for visualization and obstacle rendering.

The setup includes:

- `numpy==1.25` for stable math operations  
- `pandas` for logging and dataset handling  
- `plotly` for 3D visualization  
- `gtsam==4.2` for factor-graph optimization  
- `helpers_obstacles.py`, a custom file for drone dynamics and plotting

Once these are installed, we can proceed to path planning with RRT/RRT* and trajectory optimization.


In [ ]:
# Install GTSAM + correct numpy into an isolated folder
!pip install --upgrade --quiet pip
!pip install numpy==1.25.0 gtsam==4.2 --no-deps --target=/content/gtsam_env

# Add isolated environment to Python path
import sys
sys.path.append('/content/gtsam_env')

# Import from isolated environment
import numpy as np
import gtsam

print("GTSAM and numpy successfully loaded from isolated environment!")
print("numpy version:", np.__version__)

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import unittest


from google.colab import files
uploaded = files.upload()   # Upload helpers_obstacles.py manually


from helpers_obstacles import Drone, axes, axes_figure

print("Helper file loaded successfully!")


  Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached gtsam-4.2-cp311-cp311-manylinux2014_x86_64.whl.metadata (7.6 kB)
Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
Using cached gtsam-4.2-cp311-cp311-manylinux2014_x86_64.whl (22.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [numpy]
GTSAM and numpy successfully loaded from isolated environment!
numpy version: 2.0.2


# Part 1: Rapidly-Exploring Random Trees (RRT) in 3D

RRT is back — but now we're upgrading to full 3D motion planning!  
Instead of navigating on a 2D plane, the drone must move freely through space, around obstacles and through hoops.

### Why 3D RRT?
A drone operates in 6-DoF, but for path planning the core requirement is a feasible **(x, y, z)** path.  
So each sampled node now represents a point in space, stored as a `gtsam.Point3`.

### What we'll do in this section:
- Load the NEU drone racing track  
- Visualize the hoops suspended in mid-air  
- Generate a 3D RRT path from a start pose to a final location  
- Use GTSAM types such as `Pose3`, `Rot3`, and `Point3`

Once the track is loaded, we can attempt our first 3D RRT through the racing hoops!


In [1]:
import math
import gtsam
import helpers_obstacles as helpers  # <-- this fixes the NameError

# starting pose for the drone (45° yaw, positioned at x=1, y=3, z=8)
start_race = gtsam.Pose3(
    r=gtsam.Rot3.Yaw(math.radians(45)),
    t=gtsam.Point3(1, 3, 8)
)

# load racing hoops and simple obstacles
hoops = helpers.get_hoops()
obstacles = helpers.get_obstacles_easy()

# run RRT through the hoops (no obstacles)
helpers.drone_racing_path(hoops, start_race, [])

# run RRT through hoops + obstacles (optional)
helpers.drone_racing_path_with_obstacles(
    hoops,
    start_race,
    [],
    obstacles=obstacles
)


ModuleNotFoundError: No module named 'gtsam'

#  3D RRT Utilities

The following functions implement the basic components of a 3D Rapidly-Exploring Random Tree (RRT) planner.  
They include:

- random sampling with goal bias  
- Euclidean distance in 3D  
- nearest-node search using vectorized operations  
- steering toward a sampled point  
- the main RRT loop  
- extracting the final path from the RRT tree  

These utilities form the core of the 3D path planner the drone will use to navigate through the racing track.


In [ ]:
import math
import gtsam

# random sampler with 20% goal bias
def generate_random_point(target: gtsam.Point3) -> gtsam.Point3:
    if rng.random() < 0.2:
        return target
    return gtsam.Point3(
        rng.uniform(0, 10),
        rng.uniform(0, 10),
        rng.uniform(0, 10)
    )

# Euclidean distance between 3D points
def distance_euclidean(p1: gtsam.Point3, p2: gtsam.Point3) -> float:
    return np.linalg.norm(p2 - p1)

# find closest existing node using vectorized L2 distance
def find_nearest_node(rrt: List[gtsam.Point3], node: gtsam.Point3):
    rrt_array = np.array([np.array(p) for p in rrt])
    distances = np.linalg.norm(rrt_array - np.array(node), axis=1)
    idx = np.argmin(distances)
    return rrt[idx], idx

# move a fraction toward the target (simple steering)
def steer_naive(parent: gtsam.Point3, target: gtsam.Point3, fraction=0.2):
    return parent + fraction * (target - parent)

# main RRT loop
def run_rrt(start, target, sample_fn, steer_fn, dist_fn, nearest_fn, threshold):
    rrt = [start]
    parents = [-1]
    max_iter = 2000

    for _ in range(max_iter):
        sampled = sample_fn(target)
        nearest, idx = nearest_fn(rrt, sampled)
        new_node = steer_fn(nearest, sampled)

        rrt.append(new_node)
        parents.append(idx)

        # stop early if close to goal
        if dist_fn(new_node, target) < threshold:
            break

    return rrt, parents

# run RRT
start_rrt_3d = gtsam.Point3(1, 2, 3)
target_rrt_3d = gtsam.Point3(4, 7, 2)

rrt_3d, parents_rrt_3d = run_rrt(
    start_rrt_3d,
    target_rrt_3d,
    generate_random_point,
    steer_naive,
    distance_euclidean,
    find_nearest_node,
    threshold=0.1
)

print("Total RRT Nodes:", len(rrt_3d))
helpers.visualize_tree(rrt_3d, parents_rrt_3d, start_rrt_3d, target_rrt_3d)

# extract final path from RRT
def get_rrt_path(rrt, parents):
    path = []
    i = len(rrt) - 1
    path.append(rrt[i])

    while parents[i] != -1:
        i = parents[i]
        path.append(rrt[i])

    return path[::-1]

path_rrt_3d = get_rrt_path(rrt_3d, parents_rrt_3d)
print("Path Length:", len(path_rrt_3d))
helpers.visualize_path(path_rrt_3d, start_rrt_3d, target_rrt_3d)


# Drone Dynamics

Basic utilities for handling the drone’s orientation, force transformation, and terminal velocity.  
We use `gtsam.Rot3`, `gtsam.Point3`, and `gtsam.Pose3` to represent the pose, attitude, and forces in the navigation frame.


In [ ]:
import math
import gtsam
import numpy as np

# construct a pose example (origin + identity rotation)
position = gtsam.Point3(0, 0, 0)
attitude = gtsam.Rot3()
pose = gtsam.Pose3(r=attitude, t=position)

print("Position:", position)
print("Attitude:", attitude)
print("Pose:", pose)

# compute attitude from yaw–pitch–roll
def compute_attitude_from_ypr(yaw: float, pitch: float, roll: float) -> gtsam.Rot3:
    return gtsam.Rot3.Ypr(yaw, pitch, roll)

# compute thrust force in navigation frame
def compute_force(attitude: gtsam.Rot3, thrust: float) -> gtsam.Point3:
    # thrust is along body z-axis → take the third column of rotation matrix
    z_body_in_nav = attitude.column(2)   # direction of body z-axis expressed in N
    Fx = z_body_in_nav[0] * thrust
    Fy = z_body_in_nav[1] * thrust
    Fz = z_body_in_nav[2] * thrust
    return gtsam.Point3(Fx, Fy, Fz)

# compute terminal velocity from a force vector
def compute_terminal_velocity(force: gtsam.Point3, kd: float = 0.0425) -> gtsam.Point3:
    eps = 1e-6  # stability for near-zero components
    vx = np.sign(force[0]) * np.sqrt((abs(force[0]) + eps) / kd)
    vy = np.sign(force[1]) * np.sqrt((abs(force[1]) + eps) / kd)
    vz = np.sign(force[2]) * np.sqrt((abs(force[2]) + eps) / kd)
    return gtsam.Point3(vx, vy, vz)


class TestDroneDynamics(unittest.TestCase):

    def test_compute_attitude_from_ypr(self):
        yaw = math.radians(45)
        pitch = math.radians(30)
        roll = math.radians(60)

        expected = gtsam.Rot3(
            [0.612372, 0.612372, -0.5],
            [-0.0473672, 0.65974, 0.75],
            [0.789149, -0.435596, 0.433013]
        )

        actual = compute_attitude_from_ypr(yaw, pitch, roll)
        assert actual.equals(expected, tol=1e-2)

    def test_compute_force(self):
        attitude = gtsam.Rot3(
            [0.612372, 0.612372, -0.5],
            [-0.0473672, 0.65974, 0.75],
            [0.789149, -0.435596, 0.433013]
        )
        thrust = 20.0

        expected = gtsam.Point3(15.78, -8.71, 8.66)
        actual = compute_force(attitude, thrust)

        assert np.allclose(actual, expected, atol=1e-2)

    def test_compute_terminal_velocity(self):
        force = gtsam.Point3(15.78, -8.71, 8.66)

        expected = gtsam.Point3(19.27, -14.32, 14.27)
        actual = compute_terminal_velocity(force)

        assert np.allclose(actual, expected, atol=1e-2)


suite = unittest.TestSuite()
suite.addTest(TestDroneDynamics('test_compute_attitude_from_ypr'))
suite.addTest(TestDroneDynamics('test_compute_force'))
suite.addTest(TestDroneDynamics('test_compute_terminal_velocity'))
unittest.TextTestRunner().run(suite)
